In [18]:
import pandas as pd
import geopandas as gpd

In [19]:


replace_dict = { 'WEST ROXBURY AND ROSLINDALE' : 'Roslindale' ,
                
                'JAMAICA PLAIN' : 'Jamaica Plain',

                'ROXBURY 10B' : 'Roxbury',

                'ROXBURY 10A' : 'Roxbury',

                'ROXBURY' : 'Roxbury',

                'CHARLESTOWN' : 'Charlestown',

                'SOUTH DORCHESTER' : 'Dorchester' ,

                'NORTH END' : 'North End',

                'DOWNTOWN' : 'Downtown',

                'HYDE PARK' : 'Hyde Park',


                'SOUTH BOSTON' : 'South Boston',

                'THE NORTH END' : 'North End',

                'WEST ROXBURY' : 'West Roxbury',

                'ALLSTON/BRIGHTON' : 'Allston',

                'EAST BOSTON' : 'East Boston',

                'NORTH DORCHESTER' : 'Dorchester',


                }

In [133]:
access_score= pd.read_csv('../data/sidewalks_score_main.csv')

access_score =access_score.rename(columns = {'DISTRICT_x' : 'DISTRICT'})
access_score.columns

Index(['Unnamed: 0', 'ID', 'ramp_score', 'hazard_score', 'geometry',
       'DISTRICT', 'score', 'Fixed pinch point <36” sidewalk width',
       'Trip hazards due to tree roots', 'Trip hazards not due to tree roots'],
      dtype='object')

In [21]:
access_score = access_score.replace({'DISTRICT': replace_dict})

In [28]:
region = gpd.read_file('../data/Boston_Neighborhoods.geojson')
region = region.rename(columns = {'Name' : 'DISTRICT'})
region = region[['DISTRICT', 'geometry']]

In [134]:
access_score

,Unnamed: 0,ID,ramp_score,hazard_score,geometry,DISTRICT,score,Fixed pinch point <36” sidewalk width,Trip hazards due to tree roots,Trip hazards not due to tree roots
0,0,0,1.0,11.0,"POLYGON ((772436.3028260022 2964399.527495086,...",CHARLESTOWN,12.0,3.0,3.0,5.0
1,1,1,1.0,0.0,POLYGON ((764945.1412991732 2926518.8706450015...,SOUTH DORCHESTER,1.0,0.0,0.0,0.0
2,2,2,1.0,0.0,POLYGON ((764853.4101834297 2926759.2805972546...,SOUTH DORCHESTER,1.0,0.0,0.0,0.0
3,3,3,2.0,0.0,POLYGON ((764907.1584355086 2926824.2013991773...,SOUTH DORCHESTER,2.0,0.0,0.0,0.0
4,4,4,1.0,1.0,POLYGON ((764831.0978920907 2927057.0096605867...,SOUTH DORCHESTER,2.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...
23342,23342,23515,0.0,0.0,POLYGON ((763550.6873212492 2924072.9182754606...,HYDE PARK,0.0,0.0,0.0,0.0
23343,23343,23516,0.0,2.0,MULTIPOLYGON (((750522.6517715073 2929924.9304...,WEST ROXBURY,2.0,0.0,1.0,1.0
23344,23344,23517,0.0,0.0,"POLYGON ((747005.7523092638 2926356.796822175,...",WEST ROXBURY,0.0,0.0,0.0,0.0
23345,23345,23518,0.0,0.0,POLYGON ((776379.1387651783 2959259.1424842635...,NORTH END,0.0,0.0,0.0,0.0


In [32]:
# pd.merge(region, access_score, on = 'DISTRICT')

In [33]:
access_score.groupby('DISTRICT')['SCORE'].

,Unnamed: 0,ID,ramp_score,hazard_score,geometry,DISTRICT,score,Fixed pinch point <36” sidewalk width,Trip hazards due to tree roots,Trip hazards not due to tree roots
0,0,0,1.0,11.0,"POLYGON ((772436.3028260022 2964399.527495086,...",Charlestown,12.0,3.0,3.0,5.0
1,1,1,1.0,0.0,POLYGON ((764945.1412991732 2926518.8706450015...,Dorchester,1.0,0.0,0.0,0.0
2,2,2,1.0,0.0,POLYGON ((764853.4101834297 2926759.2805972546...,Dorchester,1.0,0.0,0.0,0.0
3,3,3,2.0,0.0,POLYGON ((764907.1584355086 2926824.2013991773...,Dorchester,2.0,0.0,0.0,0.0
4,4,4,1.0,1.0,POLYGON ((764831.0978920907 2927057.0096605867...,Dorchester,2.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...
23342,23342,23515,0.0,0.0,POLYGON ((763550.6873212492 2924072.9182754606...,Hyde Park,0.0,0.0,0.0,0.0
23343,23343,23516,0.0,2.0,MULTIPOLYGON (((750522.6517715073 2929924.9304...,West Roxbury,2.0,0.0,1.0,1.0
23344,23344,23517,0.0,0.0,"POLYGON ((747005.7523092638 2926356.796822175,...",West Roxbury,0.0,0.0,0.0,0.0
23345,23345,23518,0.0,0.0,POLYGON ((776379.1387651783 2959259.1424842635...,North End,0.0,0.0,0.0,0.0


In [40]:
top_5_percent = int(5*len(access_score)/100)
top_5_percent

top_5_percent_inaccessible_score = access_score[['ID','DISTRICT','ramp_score','hazard_score','score', 'geometry']].nlargest(top_5_percent,'score')
top_5_percent_inaccessible_score = top_5_percent_inaccessible_score.groupby('DISTRICT')['ID'].count()
top_5_percent_inaccessible_score=top_5_percent_inaccessible_score.sort_values(ascending=False)
district_count = top_5_percent_inaccessible_score.reset_index()

In [42]:
merged = pd.merge(district_count, region, on = 'DISTRICT')

In [47]:
merged['val'] = merged['ID']/merged['ID'].mean()
merged

,DISTRICT,ID,geometry,val
0,Dorchester,169,"MULTIPOLYGON (((-71.05896 42.32876, -71.05914 ...",1.737789
1,Roxbury,148,"POLYGON ((-71.09646 42.29323, -71.09633 42.293...",1.521851
2,Downtown,145,"POLYGON ((-71.05342 42.36130, -71.05391 42.361...",1.491003
3,West Roxbury,141,"POLYGON ((-71.16470 42.30384, -71.17861 42.294...",1.449871
4,East Boston,89,"POLYGON ((-70.99546 42.39394, -70.99557 42.394...",0.915167
5,North End,89,"POLYGON ((-71.05200 42.36884, -71.05219 42.368...",0.915167
6,South Boston,86,"POLYGON ((-71.06104 42.34283, -71.06102 42.342...",0.884319
7,Allston,77,"POLYGON ((-71.13517 42.34601, -71.13338 42.347...",0.791774
8,Charlestown,77,"MULTIPOLYGON (((-71.07416 42.39051, -71.07417 ...",0.791774
9,Jamaica Plain,73,"POLYGON ((-71.10499 42.32610, -71.10503 42.326...",0.750643


In [130]:
import folium
import pandas as pd
import shapely
# Load data
# cnt = gpd.GeoDataFrame(sidewalks_score).set_crs('3586').to_crs(epsg=4326)
merged = gpd.GeoDataFrame(access_score).to_crs(epsg=4326)

map = folium.Map(location=[42.2995, -71.0649], zoom_start=15)


def color_cluster_sidewalks(cnt_):
    # print(cnt_)
    print(cnt_['properties'])
    cnt1 = cnt_['properties']['cnt']
    if cnt1 >  1.5:
        return  {'fillColor': '#f10b21',  'color': '#f10b21', 'fillOpacity' : '0.5'}
    elif cnt1 > 1:
        return  {'fillColor': '#f76900', 'color': '#f76900', 'fillOpacity' : '0.5'}
    elif cnt1 > 0.76:
        return  {'fillColor': '#f4a000', 'color': '#f4a000' ,'fillOpacity' : '0.5'}
    elif cnt1 > 0.9:
        return {'fillColor': '#eccf33', 'color': '#eccf33', 'fillOpacity' : '0.5'}
    else:
        return {'fillColor': '#e1fa6e', 'color': '#e1fa6e', 'fillOpacity' : '0.5'}



def convert(row):
    t = shapely.geometry.mapping(row['geometry'])
    return {
       "type": "Feature",
  "geometry": {
    "type": row['geometry'].geom_type,
    "coordinates": t['coordinates']
  },
  "properties": {
    "cnt": row['val']
  }
    }
for index,row in merged.iterrows():

    if row['geometry']== None:
        continue
    b = folium.GeoJson(convert(row), style_function = color_cluster_sidewalks)
    b.add_child(folium.Popup(                  
                          "<br><br>Count of bad sidewalks   : " + str(row['ID'])
                         ,
                         max_width=400))
    b.add_to(map)




{'cnt': 1.737789203084833}
{'cnt': 1.5218508997429305}
{'cnt': 1.4910025706940875}
{'cnt': 1.4498714652956297}
{'cnt': 0.9151670951156813}
{'cnt': 0.9151670951156813}
{'cnt': 0.884318766066838}
{'cnt': 0.7917737789203085}
{'cnt': 0.7917737789203085}
{'cnt': 0.7506426735218509}
{'cnt': 0.6580976863753213}
{'cnt': 0.09254498714652956}


In [124]:
acc

,DISTRICT,ID_x,val,ID_y,ramp_score,hazard_score,score,geometry
0,Dorchester,169,1.737789,12081,8.0,11.0,19.0,"MULTIPOLYGON (((-71.04478 42.31882, -71.04479 ..."
1,Dorchester,169,1.737789,17307,1.0,12.0,13.0,"POLYGON ((-71.07607 42.28175, -71.07609 42.281..."
2,Dorchester,169,1.737789,12080,4.0,5.0,9.0,"MULTIPOLYGON (((-71.04486 42.31865, -71.04486 ..."
3,Dorchester,169,1.737789,12275,9.0,0.0,9.0,"POLYGON ((-71.09056 42.27210, -71.09057 42.272..."
4,Dorchester,169,1.737789,16721,1.0,8.0,9.0,"MULTIPOLYGON (((-71.05094 42.28877, -71.05111 ..."
...,...,...,...,...,...,...,...,...
1162,Roslindale,9,0.092545,21520,1.0,4.0,5.0,"MULTIPOLYGON (((-71.15557 42.28686, -71.15557 ..."
1163,Roslindale,9,0.092545,22291,1.0,4.0,5.0,"POLYGON ((-71.15250 42.26051, -71.15250 42.260..."
1164,Roslindale,9,0.092545,22313,4.0,1.0,5.0,"MULTIPOLYGON (((-71.15362 42.28835, -71.15361 ..."
1165,Roslindale,9,0.092545,23173,0.0,5.0,5.0,"MULTIPOLYGON (((-71.14962 42.28519, -71.14953 ..."


In [60]:

map.save("../maps/sidewalks/top_5_region.html")

In [118]:
acc = pd.merge(merged, access_score[['ID','DISTRICT','ramp_score','hazard_score','score', 'geometry']].nlargest(top_5_percent,'score'), on = 'DISTRICT')

In [119]:
acc = acc.drop('geometry_x', axis = 1)
acc = acc.rename(columns = {'geometry_y':'geometry'})

In [132]:
acc

,DISTRICT,ID_x,val,ID_y,ramp_score,hazard_score,score,geometry
0,Dorchester,169,1.737789,12081,8.0,11.0,19.0,"MULTIPOLYGON (((-71.04478 42.31882, -71.04479 ..."
1,Dorchester,169,1.737789,17307,1.0,12.0,13.0,"POLYGON ((-71.07607 42.28175, -71.07609 42.281..."
2,Dorchester,169,1.737789,12080,4.0,5.0,9.0,"MULTIPOLYGON (((-71.04486 42.31865, -71.04486 ..."
3,Dorchester,169,1.737789,12275,9.0,0.0,9.0,"POLYGON ((-71.09056 42.27210, -71.09057 42.272..."
4,Dorchester,169,1.737789,16721,1.0,8.0,9.0,"MULTIPOLYGON (((-71.05094 42.28877, -71.05111 ..."
...,...,...,...,...,...,...,...,...
1162,Roslindale,9,0.092545,21520,1.0,4.0,5.0,"MULTIPOLYGON (((-71.15557 42.28686, -71.15557 ..."
1163,Roslindale,9,0.092545,22291,1.0,4.0,5.0,"POLYGON ((-71.15250 42.26051, -71.15250 42.260..."
1164,Roslindale,9,0.092545,22313,4.0,1.0,5.0,"MULTIPOLYGON (((-71.15362 42.28835, -71.15361 ..."
1165,Roslindale,9,0.092545,23173,0.0,5.0,5.0,"MULTIPOLYGON (((-71.14962 42.28519, -71.14953 ..."


In [149]:
import folium
import pandas as pd
import shapely
# Load data
# cnt = gpd.GeoDataFrame(sidewalks_score).set_crs('3586').to_crs(epsg=4326)

from shapely.wkt import loads
access_score['geometry'] =  access_score['geometry'].apply(loads)
acc = gpd.GeoDataFrame(access_score).set_crs('3586').to_crs(epsg=4326)
map = folium.Map(location=[42.2995, -71.0649], zoom_start=15)
# cnt['cnt'] = cnt['score']
# ramps_0 = pd.read_csv('../data/Ramps_0_with_score.csv')
# sidewalks_hazards = pd.read_csv("../data/preprocessed_sidewalks_hazards.csv")

# sidewalks_hazards['x'] = sidewalks_hazards['x'].astype(float)
# sidewalks_hazards['y'] = sidewalks_hazards['y'].astype(float)



def color_cluster_sidewalks(cnt_):
    # print(cnt_)
    # print(cnt_['properties'])
    cnt1 = cnt_['properties']['score']
    if cnt1 >  20:
        return  {'fillColor': '#2a4858',  'color': '#2a4858', 'fillOpacity' : '0.5'}
    elif cnt1 > 11:
        return  {'fillColor': '#007882', 'color': '#007882', 'fillOpacity' : '0.5'}
    elif cnt1 > 5:
        return  {'fillColor': '#215d6e', 'color': '#215d6e' ,'fillOpacity' : '0.5'}
    elif cnt1 > 3:
        return {'fillColor': '#089f8f', 'color': '#089f8f', 'fillOpacity' : '0.5'}
    else:
        return {'fillColor': '#fafa6e', 'color': '#fafa6e', 'fillOpacity' : '0.5'}
    
brown_layer = folium.FeatureGroup(name='Score more than 10 Hazards(BROWN)')

red_layer = folium.FeatureGroup(name='Score more than 5(RED)')
orange_layer = folium.FeatureGroup(name = 'Score more than 2 less than 5(ORANGE)')
yellow_layer = folium.FeatureGroup(name='Score more than 0 less than 2(YELLOW)')
green_layer = folium.FeatureGroup(name='Score 0 (GREEN)')



def convert(row):
    t = shapely.geometry.mapping(row['geometry'])
    return {
       "type": "Feature",
  "geometry": {
    "type": row['geometry'].geom_type,
    "coordinates": t['coordinates']
  },
  "properties": {
    "score": row['score']
  }
    }
# for index,row in acc.iterrows():

#     if row['geometry']== None:
#         continue
    # folium.GeoJson(convert(row), style_function=color_cluster_sidewalks).add_to(map)
folium.GeoJson(acc, style_function = color_cluster_sidewalks ).add_to(map)
   


folium.LayerControl().add_to(map)
map.save("../maps/sidewalks/top_5_region.html")

In [142]:
acc

,Unnamed: 0,ID,ramp_score,hazard_score,geometry,DISTRICT,score,Fixed pinch point <36” sidewalk width,Trip hazards due to tree roots,Trip hazards not due to tree roots
0,0,0,1.0,11.0,"POLYGON ((-71.06966 42.38166, -71.06961 42.381...",CHARLESTOWN,12.0,3.0,3.0,5.0
1,1,1,1.0,0.0,"POLYGON ((-71.09805 42.27781, -71.09805 42.277...",SOUTH DORCHESTER,1.0,0.0,0.0,0.0
2,2,2,1.0,0.0,"POLYGON ((-71.09838 42.27847, -71.09838 42.278...",SOUTH DORCHESTER,1.0,0.0,0.0,0.0
3,3,3,2.0,0.0,"POLYGON ((-71.09818 42.27865, -71.09819 42.278...",SOUTH DORCHESTER,2.0,0.0,0.0,0.0
4,4,4,1.0,1.0,"POLYGON ((-71.09846 42.27929, -71.09846 42.279...",SOUTH DORCHESTER,2.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...
23342,23342,23515,0.0,0.0,"POLYGON ((-71.10324 42.27112, -71.10321 42.271...",HYDE PARK,0.0,0.0,0.0,0.0
23343,23343,23516,0.0,2.0,"MULTIPOLYGON (((-71.15129 42.28733, -71.15119 ...",WEST ROXBURY,2.0,0.0,1.0,1.0
23344,23344,23517,0.0,0.0,"POLYGON ((-71.16434 42.27758, -71.16434 42.277...",WEST ROXBURY,0.0,0.0,0.0,0.0
23345,23345,23518,0.0,0.0,"POLYGON ((-71.05516 42.36750, -71.05520 42.367...",NORTH END,0.0,0.0,0.0,0.0


In [112]:
acc['DISTRICT'].unique()

array(['Dorchester', 'Roxbury', 'Downtown', 'West Roxbury', 'East Boston',
       'North End', 'South Boston', 'Allston', 'Charlestown',
       'Jamaica Plain', 'Hyde Park', 'Roslindale'], dtype=object)

In [113]:
t= pd.read_csv('../data/sidewalks_score_main.csv')

In [115]:
t['DISTRICT_x'].unique()

array(['CHARLESTOWN', 'SOUTH DORCHESTER', 'NORTH END', 'DOWNTOWN',
       'HYDE PARK', 'ROXBURY 10B', 'SOUTH BOSTON', 'THE NORTH END',
       'WEST ROXBURY', 'JAMAICA PLAIN', 'ROXBURY 10A', 'ALLSTON/BRIGHTON',
       'EAST BOSTON', 'NORTH DORCHESTER', 'ROXBURY',
       'WEST ROXBURY AND ROSLINDALE', '0'], dtype=object)

In [116]:
access_score[['ID','DISTRICT','ramp_score','hazard_score','score', 'geometry']].nlargest(top_5_percent,'score')

,ID,DISTRICT,ramp_score,hazard_score,score,geometry
11249,16050,Allston,21.0,16.0,37.0,MULTIPOLYGON (((750242.4180800021 2948753.0427...
13934,22780,Allston,28.0,1.0,29.0,MULTIPOLYGON (((760709.1084326804 2953045.8103...
7651,9529,South Boston,13.0,9.0,22.0,MULTIPOLYGON (((775917.6593803465 2946912.5404...
2367,2778,West Roxbury,2.0,19.0,21.0,"POLYGON ((752162.1042012572 2931655.839045584,..."
3905,4617,Charlestown,1.0,19.0,20.0,"POLYGON ((772316.742697671 2964646.4833660126,..."
...,...,...,...,...,...,...
6561,8068,South Boston,2.0,2.0,4.0,"POLYGON ((775863.0417074263 2949876.641186595,..."
6575,8082,South Boston,2.0,2.0,4.0,"POLYGON ((776604.3250017613 2945824.123373002,..."
6577,8087,South Boston,4.0,0.0,4.0,POLYGON ((778486.7346227616 2948103.3159930855...
6600,8117,South Boston,2.0,2.0,4.0,MULTIPOLYGON (((777681.442598179 2948926.54564...


In [ ]:
import folium
import pandas as pd

# Load data
cnt = gpd.GeoDataFrame(sidewalks_score).set_crs('3586').to_crs(epsg=4326)
cnt = cnt.to_crs(epsg=4326)

cnt['cnt'] = cnt['score']
ramps_0 = pd.read_csv('../data/Ramps_0_with_score.csv')
sidewalks_hazards = pd.read_csv("../data/preprocessed_sidewalks_hazards.csv")

sidewalks_hazards['x'] = sidewalks_hazards['x'].astype(float)
sidewalks_hazards['y'] = sidewalks_hazards['y'].astype(float)
map = folium.Map(location=[42.2995, -71.0649], zoom_start=15)

# Add sidewalks layer
# merged_sidewalk_hazards['centroids']= gpd.GeoSeries.from_wkt(merged_sidewalk_hazards['centroids'])
# merged_sidewalk_hazards = gpd.GeoDataFrame(merged_sidewalk_hazards)
# merged_sidewalk_hazards= merged_sidewalk_hazards.set_geometry('centroids')
# merged_sidewalk_hazards.crs = dest_epsg
# merged_sidewalk_hazards = merged_sidewalk_hazards.to_crs('4326')

def color_cluster_sidewalks(cnt_):
    # print(cnt_)
 
    cnt1 = cnt_['properties']['cnt']
    if cnt1 > 20:
        return  {'fillColor': 'brown', 'color': 'brown'}
    elif cnt1 > 11:
        return  {'fillColor': 'red', 'color': 'red'}
    elif cnt1 > 6:
        return  {'fillColor': 'orange', 'color': 'orange'}
    elif cnt1 > 3:
        return {'fillColor': 'yellow', 'color': 'yellow'}
    else:
        return {'fillColor': 'green', 'color': 'green'}
    
brown_layer = folium.FeatureGroup(name='Score more than 10 Hazards(BROWN)')

red_layer = folium.FeatureGroup(name='Score more than 5(RED)')
orange_layer = folium.FeatureGroup(name = 'Score more than 2 less than 5(ORANGE)')
yellow_layer = folium.FeatureGroup(name='Score more than 0 less than 2(YELLOW)')
green_layer = folium.FeatureGroup(name='Score 0 (GREEN)')



def convert(row):
    t = shapely.geometry.mapping(row['geometry'])
    return {
       "type": "Feature",
  "geometry": {
    "type": row['geometry'].geom_type,
    "coordinates": t['coordinates']
  },
  "properties": {
    "cnt": row['cnt']
  }
    }
for index,row in cnt.iterrows():

    if row['geometry']== None:
        continue
    b = folium.GeoJson(convert(row), style_function = color_cluster_sidewalks)
    b.add_child(folium.Popup("Index                       : " + str(index) + "<br>Score: " + str(row['cnt'])
                         + "<br><br>Fixed pinch point     : " + str(row['Fixed pinch point <36” sidewalk width'])
                         + "<br><br>Trip due to tree roots: " + str(row['Trip hazards due to tree roots'])
                         + "<br><br>Trip due to cracks: " + str(row['Trip hazards not due to tree roots'])
                          + "<br><br>Bad Ramps: " + str(row['ramp_score']),
                         max_width=400))
    if row['cnt'] > 10:
        b.add_to(brown_layer)
    elif row['cnt'] > 5:
        b.add_to(red_layer)
    elif row['cnt'] > 2:
        b.add_to(orange_layer)
    elif row['cnt'] > 0:
        b.add_to(yellow_layer)
    else:
        b.add_to(green_layer)


red_layer.add_to(map)
orange_layer.add_to(map)
yellow_layer.add_to(map)
green_layer.add_to(map)
brown_layer.add_to(map)




folium.LayerControl().add_to(map)
map.save("../maps/sidewalks/sidewalks_with_score.html")